In [1]:
# !conda install pandas numpy scipy ipython -y
# !conda install -c maciejkula -c pytorch spotlight -y
# !pip install spotlight

!conda install pytorch torchvision  cudatoolkit=10.2 -c pytorch --force-reinstall -y

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\ProgramData\Anaconda3\envs\pytorch

  added / updated specs:
    - cudatoolkit=10.2
    - pytorch
    - torchvision



Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done


In [2]:
import sys
# sys.path.append("../../")
import os
# import papermill as pm

import pandas as pd
import numpy as np
# import tensorflow as tf
# from reco_utils.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k, get_top_k_items, auc

from sklearn.model_selection import train_test_split, GroupShuffleSplit, GridSearchCV, GroupKFold
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from spotlight.cross_validation import random_train_test_split, user_based_train_test_split
from spotlight.datasets.movielens import get_movielens_dataset
from spotlight.evaluation import mrr_score
from spotlight.factorization.implicit import ImplicitFactorizationModel
from spotlight.cross_validation import user_based_train_test_split
from spotlight.datasets.synthetic import generate_sequential
from spotlight.evaluation import sequence_mrr_score
from spotlight.sequence.implicit import ImplicitSequenceModel
from spotlight.interactions import Interactions
import torch
from spotlight.evaluation import sequence_mrr_score,precision_recall_score
# from spotlight.evaluation import sequence_precision_recall_score # https://maciejkula.github.io/spotlight/evaluation.html#spotlight.evaluation.sequence_precision_recall_score

In [3]:
torch.version.cuda

'8.0'

In [4]:
def sequence_precision_recall_score(model, test, k=10, exclude_preceding=False):
    """
    https://maciejkula.github.io/spotlight/_modules/spotlight/evaluation.html#sequence_precision_recall_score
    
    Compute sequence precision and recall scores. Each sequence
    in test is split into two parts: the first part, containing
    all but the last k elements, is used to predict the last k
    elements.

    Parameters
    ----------

    model: fitted instance of a recommender model
        The model to evaluate.
    test: :class:`spotlight.interactions.SequenceInteractions`
        Test interactions.
    exclude_preceding: boolean, optional
        When true, items already present in the sequence will
        be excluded from evaluation.

    Returns
    -------

    mrr scores: numpy array of shape (num_users,)
        Array of MRR scores for each sequence in test.
    """
    sequences = test.sequences[:, :-k]
    targets = test.sequences[:, -k:]
    precision_recalls = []
    for i in range(len(sequences)):
        predictions = -model.predict(sequences[i])
        if exclude_preceding:
            predictions[sequences[i]] = FLOAT_MAX

        predictions = predictions.argsort()[:k]
        precision_recall = _get_precision_recall(predictions, targets[i], k)
        precision_recalls.append(precision_recall)

    precision = np.array(precision_recalls)[:, 0]
    recall = np.array(precision_recalls)[:, 1]
    return precision, recall


def _get_precision_recall(predictions, targets, k):

    predictions = predictions[:k]
    num_hit = len(set(predictions).intersection(set(targets)))

    return float(num_hit) / len(predictions), float(num_hit) / len(targets)


In [5]:
# top k items to recommend
TOP_K = 4

# Model parameters
EPOCHS = 10
BATCH_SIZE =  256#1024

SEED = 0  # Set None for non-deterministic results

user_file = "../../tests/resources/deeprec/lightgcn/user_embeddings.csv"
item_file = "../../tests/resources/deeprec/lightgcn/item_embeddings.csv"

TRAIN_FILE_PATH = os.path.normpath(r"C:\Users\Dan Ofer\Desktop\Stuff\booking_wisdom\booking_train_set.csv") #"/content/drive/MyDrive/booking_wisdom/booking_train_set.csv" #"booking_train_set.csv"


Spotlight relevant examples:

https://github.com/maciejkula/spotlight/blob/master/examples/movielens_sequence/movielens_sequence.py

https://github.com/maciejkula/spotlight/issues/172
* Dataset creation/loading + get predictions (sequence)

Spotlight hyperparameter tuning:
https://github.com/maciejkula/spotlight/blob/master/examples/movielens_sequence/movielens_sequence.py

In [6]:
# df = movielens.load_pandas_df(size=MOVIELENS_DATA_SIZE) ## orig 
# print(df.dtypes)
# print(df.shape)
# ## expected columns: ['userID', 'itemID', 'rating', 'timestamp'] (last as float), others as number
# df.head()

In [7]:
# df = movielens.load_pandas_df(size=MOVIELENS_DATA_SIZE) ## orig 
df = pd.read_csv(TRAIN_FILE_PATH,
                     nrows=7_500,
                     parse_dates=["checkin"],infer_datetime_format=True,usecols=["utrip_id","city_id","checkin"])
user_encoder = LabelEncoder()


## drop consecutive elements (need to do within group, and to order, and what about updating checkout date?)
## 0 cases if we also use checkin or checkout. 
## The "last"  criteria does drop rows! Do we want or not want it???
## We also +- need to merge with total duration data

df["utrip_id"] = user_encoder.fit_transform(df["utrip_id"])
df = df.groupby("utrip_id").filter(lambda x: len(x) >= 4) # keep only trips with at least 4

#####
df["row_num"] = df.groupby("utrip_id")["checkin"].rank(ascending=True,pct=False).astype(int)
utrip_counts = df["utrip_id"].value_counts()
df["total_rows"] = df["utrip_id"].map(utrip_counts)
print("max rows", df["total_rows"].describe())
df["last"] = (df["row_num"] ==df["total_rows"])#.astype(int)
###
df = df.loc[(df[["city_id","utrip_id","last"]].shift() != df[["city_id","utrip_id","last"]]).max(axis=1)]

df = df[["utrip_id","city_id","checkin","last"]]
              
df.columns = ['userID', 'itemID',  'timestamp',"last"] ## no rating col

df.sort_values(["userID","timestamp"],ascending=True,inplace=True) ## reinforce sort order for within groups
# df["rating"] = 1

df = df[['userID', 'itemID',  'timestamp']]
# print(df.dtypes)
print("nunique\n",df.nunique())
print(df.shape)
df.head()

max rows count    7496.000000
mean        6.045624
std         2.612190
min         4.000000
25%         4.000000
50%         5.000000
75%         7.000000
max        23.000000
Name: total_rows, dtype: float64
nunique
 userID       1399
itemID       2895
timestamp     419
dtype: int64
(7012, 3)


,userID,itemID,timestamp
5725,0,61586,2016-10-22
5726,0,15990,2016-10-24
5727,0,28273,2016-10-26
5728,0,47486,2016-10-28
0,1,31114,2016-04-09


In [8]:
print("unique items (city_id)")
print(df['itemID'].nunique())
df['itemID'].value_counts().describe()

unique items (city_id)
2895


count    2895.000000
mean        2.422107
std         4.529493
min         1.000000
25%         1.000000
50%         1.000000
75%         2.000000
max        62.000000
Name: itemID, dtype: float64

In [9]:
### replace rare variables (under 2 occurrences) with "-1" dummy OR with their hotel country! (as negative mapped number)
city_ids_counts = df["itemID"].value_counts()
print("before:", city_ids_counts)
print("uniques",df["itemID"].nunique())
city_ids_counts = city_ids_counts.to_dict()
# df["itemID"] = df["itemID"].where(df["itemID"].apply(lambda x: x.map(x.value_counts()))>=3, -1)
df["itemID"] = df["itemID"].where(df["itemID"].map(city_ids_counts)>2, -1)

## run encoder on cities (due to negative and to ensure ok ordering for spotlight)
item_encoder = LabelEncoder()
df["itemID"] = user_encoder.fit_transform(df["itemID"])

print("unique items (city_id)")
print(df['itemID'].nunique())
print(df['itemID'].value_counts().describe())

before: 47499    62
17127    61
23921    59
55128    53
29770    53
         ..
53890     1
27862     1
32458     1
64000     1
34815     1
Name: itemID, Length: 2895, dtype: int64
uniques 2895
unique items (city_id)
565
count     565.000000
mean       12.410619
std       116.150083
min         3.000000
25%         3.000000
50%         5.000000
75%         8.000000
max      2761.000000
Name: itemID, dtype: float64


In [10]:
# df.loc[~(df[["city_id","utrip_id"]].shift() != df[["city_id","utrip_id"]]).max(axis=1)]
# df.loc[~(df[["city_id","utrip_id","last"]].shift() != df[["city_id","utrip_id","last"]]).max(axis=1)]

In [11]:
%%time
# ## Original:
# train, test = python_stratified_split(df, ratio=0.75)

Wall time: 0 ns


* Create spotlight Interactions dataset
    * https://github.com/maciejkula/spotlight/issues/172
    * Could provide WEIGHTS (more weight to final row in sequence / last) 
    
    * Getting predictions: 
    ```
    predictions = model.predict(ids)

    item_ids= (-predictions).argsort()[:10] # last 10 items
    print(item_ids)
    print(predictions[item_ids])
    ```

In [13]:
implicit_interactions = Interactions(df['userID'].values, 1+df['itemID'].values, timestamps=df['timestamp'])

train, test = user_based_train_test_split(implicit_interactions, 0.25) # ,random_state=
print("train",train)
print("test",test)


train <Interactions dataset (1402 users x 566 items x 5146 interactions)>
test <Interactions dataset (1402 users x 566 items x 1866 interactions)>


In [ ]:
# # OLD: manual train test split - only last item , for some users

train_inds, test_inds = next(GroupShuffleSplit(test_size=.35, ## test_size=.35, # when getting only last
                                               n_splits=2,
                                               random_state = 0).split(df,groups=df['userID']))
# X_train = df.iloc[train_inds]
X_test = df.iloc[test_inds]

train = pd.concat([df.iloc[train_inds],
                  X_test.loc[X_test["last"]==False]]).drop(["last"],axis=1)

test = pd.concat([X_test.loc[X_test["last"]==True]]).drop(["last"],axis=1)

train.sort_values(["userID","timestamp"],ascending=True,inplace=True)
test.sort_values(["userID","timestamp"],ascending=True,inplace=True)

print(train.shape)
print(test.shape)

In [ ]:
%%time
sequential_interaction = train.to_sequence(max_sequence_length=9, min_sequence_length=3)
print(sequential_interaction)
test_sequential = test.to_sequence(max_sequence_length=16, min_sequence_length=4)
print(test_sequential)

In [ ]:
%%time
implicit_sequence_model = ImplicitSequenceModel(use_cuda=False, ##  error when running locally with pytorch - using wrong cuda version - dan windows
                                                n_iter=5, 
                                                loss='bpr', # 'pointwise', 'bpr', 'hinge', 'adaptive_hinge'
                                                representation='lstm', # 'pooling', 'cnn', 'lstm', 'mixture'
                                               batch_size=256,
                                               embedding_dim=64) #  , sparse=True
implicit_sequence_model.fit(sequential_interaction, verbose=True)

In [ ]:
# precision_recall_score(implicit_sequence_model, test, k=4)

In [ ]:
## https://maciejkula.github.io/spotlight/evaluation.html#spotlight.evaluation.sequence_precision_recall_score
## listed in spotlight docs, not found on import ? 
# Compute sequence precision and recall scores. 
# Each sequence in test is split into two parts: the first part, containing all but the last k elements, is used to predict the last k elements.


### returns score per user
seq_pr_score_array = sequence_precision_recall_score(implicit_sequence_model, test_sequential, k=1, exclude_preceding=False) 

print("pr_score_array[0].mean",seq_pr_score_array[0].mean())
print("pr_score_array[1].mean",seq_pr_score_array[1].mean()) ## the 2 arrays appear identical  ? 
seq_pr_score_array

In [ ]:
# (seq_pr_score_array[0] ==seq_pr_score_array[1]).mean() ## 1.0 

In [ ]:
seq_pr_score_array[1].mean()

## Dan: 
### Below this is old code from the ms recomenders notebook, not yet updated for new evaluation. Ignopre it for now! 

### 3.4 Recommendation and Evaluation

Recommendation and evaluation have been performed on the specified test set during training. After training, we can also use the model to perform recommendation and evalution on other data. Here we still use `test` as test data, but `test` can be replaced by other data with similar data structure.

#### 3.4.1 Recommendation

We can call `recommend_k_items` to recommend k items for each user passed in this function. 
(Originally - had set `remove_seen=True` to remove the items already seen by the user).
The function returns a dataframe, containing each user and top k items recommended to them and the corresponding ranking scores.

In [ ]:
## get last row per user id (i.e final destination) - for evaluating predictions on last destination
# warning - existing train test split is random, not for last.. model isn';t time aware! 
test_last = test.groupby("userID").last().reset_index()[["userID","itemID"]]
assert test["userID"].nunique() == test_last.shape[0]
test_last

In [ ]:
topk_scores = model.recommend_k_items(test, top_k=TOP_K+1,
                                      remove_seen=False)

## drop the rare values (we replaced with negatives) -since we don't know them. take next best prediction, then keep only top 4
topk_scores = topk_scores.loc[topk_scores["itemID"]>=0] 
## keep top 4
topk_scores = topk_scores.groupby("userID").head(4)#.reset_index()

topk_scores
topk_scores.head()

In [ ]:
topk_merged = topk_scores.merge(test_last,on=["userID","itemID"],how="right")#.fillna(0) # nan/0 = not predicted
display(topk_merged)
print(topk_merged.isna().sum())
print("Topk Accuracy: %.3f" % (100*(1-(topk_merged["prediction"].isna().sum())/topk_merged["userID"].nunique())))

#### 3.4.2 Evaluation

With `topk_scores` predicted by the model, we can evaluate how LightGCN performs on this test set.

In [ ]:
eval_map = map_at_k(test, topk_scores, k=TOP_K)
eval_ndcg = ndcg_at_k(test, topk_scores, k=TOP_K)
eval_precision = precision_at_k(test, topk_scores, k=TOP_K)
eval_recall = recall_at_k(test, topk_scores, k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

### 3.5 Infer embeddings

With `infer_embedding` method of LightGCN model, we can export the embeddings of users and items in the training set to CSV files for future use.

In [ ]:
# model.infer_embedding(user_file, item_file)

### 3.6 Compare with SAR and NCF

Here there are the performances of LightGCN compared to [SAR](../00_quick_start/sar_movielens.ipynb) and [NCF](../00_quick_start/ncf_movielens.ipynb) on MovieLens dataset of 100k and 1m. The method of data loading and splitting is the same as that described above and the GPU used was a GeForce GTX 1080Ti.

Settings common to the three models: `epochs=15, seed=42`.

Settings for LightGCN: `embed_size=64, n_layers=3, batch_size=1024, decay=0.0001, learning_rate=0.015 `.

Settings for SAR: `similarity_type="jaccard", time_decay_coefficient=30, time_now=None, timedecay_formula=True`.

Settings for NCF: `n_factors=4, layer_sizes=[16, 8, 4], batch_size=1024, learning_rate=0.001`.

| Data Size | Model    | Training time | Recommending time | MAP@10   | nDCG@10  | Precision@10 | Recall@10 |
| --------- | -------- | ------------- | ----------------- | -------- | -------- | ------------ | --------- |
| 100k      | LightGCN | 27.8865       | 0.6445            | 0.129236 | 0.436297 | 0.381866     | 0.205816  |
| 100k      | SAR      | 0.4895        | 0.1144            | 0.110591 | 0.382461 | 0.330753     | 0.176385  |
| 100k      | NCF      | 116.3174      | 7.7660            | 0.105725 | 0.387603 | 0.342100     | 0.174580  |
| 1m        | LightGCN | 396.7298      | 1.4343            | 0.075012 | 0.377501 | 0.345679     | 0.128096  |
| 1m        | SAR      | 4.5593        | 2.8357            | 0.060579 | 0.299245 | 0.270116     | 0.104350  |
| 1m        | NCF      | 1601.5846     | 85.4567           | 0.062821 | 0.348770 | 0.320613     | 0.108121  |

From the above results, we can see that LightGCN performs better than the other two models.

### Reference: 
1. Xiangnan He, Kuan Deng, Xiang Wang, Yan Li, Yongdong Zhang & Meng Wang, LightGCN: Simplifying and Powering Graph Convolution Network for Recommendation, 2020, https://arxiv.org/abs/2002.02126

2. LightGCN implementation [TensorFlow]: https://github.com/kuandeng/lightgcn